In [226]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append(os.path.abspath("../src"))  # path to the folder containing helper.py

from QPU import *
from Qint import *
from Shor import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [227]:
qpu = QPU()
qint1 = qpu.allocate_qubits(6)
qint2 = qpu.allocate_qubits(5, type="address")
qint1.overflow = False
for i in range(2**len(qint2)):
    qpu.data[i] = 0
print(qint1.start)
print(qint2.start)
print(qpu.data)

5
0
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0}


In [228]:
print(len(qint1))          # should print 5
# print(qint1[:3])           # should return a Qint of length 3
print(qint1[:3].start)     # check start index
print(qint1[:3].length)    # check length


6
5
3


In [229]:
q_add = qint2
print(q_add.start)
print(q_add.data)
table = {}
import random
for i in range(2**len(q_add)):
    table[i] = random.getrandbits(len(q_add)) 
lookup = LookupTable(q_add, table)

0
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0}


In [230]:
print(len(table.keys()))
qint1 += lookup
print(qint1)
record = qint1._parent.data.copy()

32
|000010|00000 has amp 0
|010101|00001 has amp 0
|010000|00010 has amp 0
|000110|00011 has amp 0
|011100|00100 has amp 0
|001100|00101 has amp 0
|001111|00110 has amp 0
|000111|00111 has amp 0
|001010|01000 has amp 0
|010110|01001 has amp 0
|010000|01010 has amp 0
|010111|01011 has amp 0
|010010|01100 has amp 0
|010000|01101 has amp 0
|000010|01110 has amp 0
|000100|01111 has amp 0
|011010|10000 has amp 0
|011111|10001 has amp 0
|010101|10010 has amp 0
|011001|10011 has amp 0
|010110|10100 has amp 0
|010011|10101 has amp 0
|001101|10110 has amp 0
|010010|10111 has amp 0
|010011|11000 has amp 0
|010110|11001 has amp 0
|011010|11010 has amp 0
|011100|11011 has amp 0
|001010|11100 has amp 0
|001000|11101 has amp 0
|000010|11110 has amp 0
|011011|11111 has amp 0


In [ ]:
p_len = 3
module = random.getrandbits(p_len)
while module < 2**(p_len-1):
    module = random.getrandbits(p_len)
print(bin(module<<5))
print(qint1.start)
original_data = qpu.data.copy()
reduced_size = len(qint1) - module.bit_length() - 1
loop2(qint1, qint2, module, p_len)
print(qint1)


0b10000000
5
|0010|00000 has amp 0
|0001|00001 has amp 0
|0000|00010 has amp 0
|0010|00011 has amp 0
|0000|00100 has amp 0
|0000|00101 has amp 0
|0011|00110 has amp 0
|0011|00111 has amp 0
|0010|01000 has amp 0
|0010|01001 has amp 0
|0000|01010 has amp 0
|0011|01011 has amp 0
|0010|01100 has amp 0
|0000|01101 has amp 0
|0010|01110 has amp 0
|0000|01111 has amp 0
|0010|10000 has amp 0
|0011|10001 has amp 0
|0001|10010 has amp 0
|0001|10011 has amp 0
|0010|10100 has amp 0
|0011|10101 has amp 0
|0001|10110 has amp 0
|0010|10111 has amp 0
|0011|11000 has amp 0
|0010|11001 has amp 0
|0010|11010 has amp 0
|0000|11011 has amp 0
|0010|11100 has amp 0
|0000|11101 has amp 0
|0010|11110 has amp 0
|0011|11111 has amp 0


In [232]:
print(qpu.phase_up)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0}


In [233]:
from toolkit import extract_bits
def test_loop2(Q_target, orginal_data, module, reduced_size):
    qpu = Q_target._parent
    for i in range(len(qpu.data.keys())):
        value = list(qpu.data.keys())[i]
        target_value = extract_bits(value, Q_target.start, Q_target.start+len(Q_target)-1)
        original_value = list(orginal_data.keys())[i]
        original_value = extract_bits(original_value, Q_target.start, Q_target.start+len(Q_target)-1 + reduced_size)
        assert original_value % module == target_value
    print("test passed")

In [234]:

print(test_loop2(qint1, original_data, module, reduced_size))

test passed
None


In [235]:
print(bin(96))
a =  s = f"{96:0{10}b}"
print(a)
print(a[4])

0b1100000
0001100000
1


In [236]:
n = 0b110111101011  # 0xDEB = 3563

# Extract bits 3 to 10
result = extract_bits(n, 2, 2)
print(bin(result))  # 0b11011110
print(result)       # 222
0b10111101

0b0
0


189

In [237]:
test = True
a = 10
print(0+test)

1


In [238]:
print(module)

4


In [239]:
print(bin(module<<5),"offset")
qint1 -= (module<<5)

0b10000000 offset


In [240]:
a = 1<<3
b = 0b0110
c = a & b
print(c)
bit_sum = c.bit_count()   # Python 3.10+


0


In [241]:
print(10&0)

0
